In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
stock_code = pd.read_csv('./stock_codes.csv')

In [3]:
def url_to_df(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # 문제가 발생하면 예외 발생
        html = response.text
    except requests.exceptions.RequestException as e:
        print(f"웹 페이지 요청 오류: {e}")
        exit()

    soup = BeautifulSoup(html, 'html.parser')

    table = soup.find('table', 'us_table_ty1 h_fix zigbg_no')

    # 테이블 데이터 추출
    if table:
        rows = table.find_all('tr')
        data = []
        for row in rows:
            cols = row.find_all(['td', 'th'])
            # 각 행의 데이터 추출
            row_data = [tag.get('title').strip() if tag.get('title') else tag.text.strip() for tag in cols]
            data.append(row_data)

    else:
        return None

    # "계산에 참여한 계정 펼치기" 없애기
    for i in range(len(data)):
        if "펼치기" in data[i][0]:
            data[i][0] = data[i][0][:-14]


    df = pd.DataFrame(data)
    df_col = pd.DataFrame(data, columns = df.iloc[0]).reset_index(drop=True)

    df_col = df_col.drop(index=0, axis=0).reset_index(drop=True)
    df_col = df_col.drop(columns=['전년동기(%)'], axis=1).reset_index(drop=True)

    df_col = df_col.replace('',0.0)

    return(df_col)

In [ ]:

# 웹 페이지 URL stock코드 불러오기
for i in tqdm(range(681,len(stock_code))):
    url = "https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701"%(stock_code['단축코드'][i])

    return_df = url_to_df(url)

    # 비어있는 표가 있으면 예외 처리
    if return_df is None:
        continue

    return_df.to_csv(f"./stocks_data/{stock_code['단축코드'][i]}_{stock_code['한글 종목명'][i]}.csv", index=False)
